<a href="https://colab.research.google.com/github/ZotovNV/Resume_for_chek/blob/main/Python_%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B_%D0%B2%D0%B5%D0%B1_%D1%81%D0%BA%D1%80%D0%B0%D0%BF%D0%B8%D0%BD%D0%B3%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup

In [ ]:
# Как показывали через 2 функции. Пока что по одной странице.

def get_habr_pub(query):
    habr_pub = pd.DataFrame()
    for q in query:
        url = 'https://habr.com/ru/search/'
        a = url.split('/ru')
        params = {
            'q': q
        }
        req = requests.get(url, params=params)
        time.sleep(0.3)
        soup = BeautifulSoup(req.text)
        articles = soup.find_all('article', class_='tm-articles-list__item')
        for article in articles:
            title = article.find('h2', class_='tm-title tm-title_h2').text
            link = a[0] + (article.find('a', class_='tm-title__link').get('href'))
            date = article.find('time').get('title')
            votes = article.find('div', class_='tm-votes-meter tm-data-icons__item').text
            row = {'date': date, 'title': title, 'link': link, 'votes': votes}
            habr_pub = pd.concat([habr_pub, pd.DataFrame([row])])
    return habr_pub.reset_index(drop=True)

result = get_habr_pub(['python'])
result

In [ ]:
def add_full_text(posts_df):
    i = 0
    for el in posts_df['link']:
        # print(el)
        req = requests.get(el).text
        soup = BeautifulSoup(req)
        time.sleep(0.3)
        text = soup.find_all('div', class_='tm-article-body')
        for ar in text:
                full_text = ar.find('div', xmlns ='http://www.w3.org/1999/xhtml').text
        posts_df.loc[i, 'text'] = full_text
        i += 1
    return posts_df

add_full_text(result)

Это второй вариант решения, с коментариями мелкими:

In [ ]:
# поступил своеобразно, т.к. каждый проход мы извлекаем ссылку, так почему бы сразу не вытаскивать и текст по этой ссылке. Осталось прикрутить страницы, но с ними сложность вижу что есть в ссылке такое
# https://habr.com/ru/search/page1/?q=python&target_type=posts&order=relevance меня интересует page1, но как передавать в параметры так и не понял 'page': i ну не работает). Я так понимаю из за того что за место page
#  ставлю число, а надо как то через конкатенацию что ли 'page':'page' + 'i'.
# По логике по началу должен быть цикл на сам текст запроса, потом цикл по числу, ну и окончательное это цикл по каждой новости(в ней же завернут цикл по извлечению полного текста). В общем то не очень хорошо, то что цикл в цикле на цикле =D.
# Но захотелось глянуть)
# Если смотреть через первый вариант решения, то там сложновато все...Последним будет прогоняться функция которая полный текст извлекает. Не вижу варианта сделать в три функции. Хоть и вариант должен быть.

def get_habr_pub(query):
    habr_pub = pd.DataFrame()
    for q in query:
        url = 'https://habr.com/ru/search/'
        a = url.split('/ru')
        params = {
            'q': q
        }
        req = requests.get(url, params=params)
        time.sleep(0.3)
        soup = BeautifulSoup(req.text)
        articles = soup.find_all('article', class_='tm-articles-list__item')
        for article in articles:
            title = article.find('h2', class_='tm-title tm-title_h2').text
            link = a[0] + (article.find('a', class_='tm-title__link').get('href'))
            date = article.find('time').get('title')
            votes = article.find('div', class_='tm-votes-meter tm-data-icons__item').text
            text_pub = (BeautifulSoup(requests.get(link).text)).find_all('div', class_='tm-article-body')
            for ar in text_pub:
                text = ar.find('div', xmlns ='http://www.w3.org/1999/xhtml').text
            row = {'date': date, 'title': title, 'link': link, 'votes': votes, 'text':text}
            habr_pub = pd.concat([habr_pub, pd.DataFrame([row])])
    return habr_pub.reset_index(drop=True)

get_habr_pub(['python', 'Анализ данных'])

In [ ]:
# а это просто шаблончик того что происходит тут << text_pub = (BeautifulSoup(requests.get(link).text)).find_all('div', class_='tm-article-body') >>
url_lib = 'https://habr.com/ru/news/531402/'

rez = requests.get(url_lib)
souz = BeautifulSoup(rez.text)
art = souz.find_all('div', class_='tm-article-body')
#print(art)
for ar in art:
    text = ar.find('div', xmlns ='http://www.w3.org/1999/xhtml').text
    print(text)

In [ ]:
# проход по страницам, от 1 до 4. можно в функцию вставить input и указать там кол-во финальное страниц для сбора информации.
def get_habr_pub(query):
    habr_pub = pd.DataFrame()
    for q in query:
        url = 'https://habr.com/ru/search/'
        a = url.split('/ru')
        for page in range(1, 4):
            params = {
                'q': q,
                'page': page
            }
            req = requests.get(url, params=params)
            time.sleep(0.3)
            soup = BeautifulSoup(req.text)
            articles = soup.find_all('article', class_='tm-articles-list__item')
            for article in articles:
                title = article.find('h2', class_='tm-title tm-title_h2').text
                link = a[0] + (article.find('a', class_='tm-title__link').get('href'))
                date = article.find('time').get('title')
                votes = article.find('div', class_='tm-votes-meter tm-data-icons__item').text
                text_pub = (BeautifulSoup(requests.get(link).text)).find_all('div', class_='tm-article-body')
                for ar in text_pub:
                    text = ar.find('div', xmlns ='http://www.w3.org/1999/xhtml').text
                row = {'date': date, 'title': title, 'link': link, 'votes': votes, 'text':text}
                habr_pub = pd.concat([habr_pub, pd.DataFrame([row])])
    return habr_pub.reset_index(drop=True)

get_habr_pub(['python'])

# В этом случае функция будет запрашивать результаты с первых трех страниц поиска и объединять их в один датафрейм. Если нужно получить результаты с большего количества страниц, можно изменить диапазон цикла "for page in range(1, 4)" на нужный.